Import libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from scipy.stats import binned_statistic_2d as histo2D

Read the files into pandas dataframes

In [3]:
path = 'E:Koulu/data/hairiopv.csv'

df = pd.read_csv(path)
df['datetime'] = pd.to_datetime(df['datetime'])
df

,datetime,gdlat,glon,blrmvd
0,2023-02-27 10:10:00,69.836006,14.645379,1.837944
1,2023-02-27 10:10:00,69.778137,14.649814,1.550324
2,2023-02-27 10:10:00,69.762451,14.606965,1.547944
3,2023-02-27 10:10:00,69.689003,14.570487,1.627104
4,2023-02-27 10:15:00,69.631760,14.574980,1.888430
...,...,...,...,...
2392521,2023-02-27 06:50:00,70.955299,33.764091,-0.618356
2392522,2023-02-27 06:50:00,70.973076,33.731205,-0.920769
2392523,2023-02-27 06:50:00,71.007675,33.746216,-0.656365
2392524,2023-02-27 06:50:00,71.077103,33.775875,-0.383100


In [4]:
lats=[59, 71]
lons=[19, 32]
res = [0.15,0.7]
frames = sorted(df['datetime'].unique())

In [5]:
i=1

for frame in frames:

    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'projection': ccrs.PlateCarree()})

    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.set_xticks(np.arange(lons[0], lons[1], 1))
    ax.set_yticks(np.arange(lats[0], lats[1]))
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_xlim(lons[0], lons[1])
    ax.set_ylim(lats[0], lats[1])
        
    df0 = df.loc[df['datetime'] == frame]

    ax.set_title(f'Baseline removed VTEC at {frames[0]}')
    statistic, x_edges, y_edges, _ = histo2D(
        df0['glon'], df0['gdlat'], df0['blrmvd'], statistic='mean', 
        bins=[np.arange(lons[0], lons[1] + res[0], res[0]), 
        np.arange(lats[0], lats[1] + res[0], res[0])]) # type: ignore

    X, Y = np.meshgrid((x_edges[:-1] + x_edges[1:]) / 2, (y_edges[:-1] + y_edges[1:]) / 2)

    x_flat = X.flatten()
    y_flat = Y.flatten()
    z_flat = statistic.flatten()

    valid = ~np.isnan(z_flat)
    x_valid = x_flat[valid]
    y_valid = y_flat[valid]
    z_valid = z_flat[valid]

    statistic, x_edges, y_edges, _ = histo2D(
            x_valid, y_valid, z_valid, statistic='mean', 
            bins=[np.arange(lons[0], lons[1] + res[1], res[1]), 
                    np.arange(lats[0], lats[1] + res[1], res[1])]) # type: ignore
    X, Y = np.meshgrid((x_edges[:-1] + x_edges[1:]) / 2, (y_edges[:-1] + y_edges[1:]) / 2)

    contour = ax.pcolormesh(X, Y, statistic.T, cmap='plasma', vmin=-2, vmax=2, 
                                transform=ccrs.PlateCarree())
    
    fig.subplots_adjust(right=0.8)
    cax = fig.add_axes([0.85, 0.155, 0.05, 0.67]) # type: ignore

    cbar = plt.colorbar(contour, cax=cax, orientation='vertical',
                            extendrect = True, label='VTEC',
                            ticks=np.arange(-2,2.5,0.5),
                            boundaries = np.arange(-2,2.5,0.5), 
                            extend='both'
        )

    plt.savefig(fname=f'ready/esim/pic{i}.png')
    i+=1
    plt.close()